# Watson OpenScale Fairness Metrics and Transformers

## 1. Introduction <a name="introduction"></a>
The notebook will train a German Credit Risk model, compute Fairness Metrics **Statistical Parity Difference** and **Smoothed Empirical Differential** on the model prediction and then show how **Fair Score Transformer** can be used to transform the model output for fair prediction.<br/>

This document includes below sections, you will *`edit`* and *`restart`* notebook kernel in **Setup** section:

- [1.Introduction](#introduction)
- [2.Setup Envrionments](#setup)
- [3.Statistical Parity Difference](#spd)
- [4.Smoothed Empirical Differential](#sed)
- [5.Fair Score Transformer](#fst)

**Note:** This notebook should be run using with **Python 3.9.x** runtime. It requires service credentials for the following services:
  * Watson OpenScale <br/>
  * IBM Analytics Engine <br/>

## 2. Setup Envrionments <a name="setup"></a>

### 2.1 Package installation

*[Optional]* ignore warning messages to make output more clear.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

Install packages:

In [ ]:
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1
!pip install --upgrade ibm-metrics-plugin --user | tail -n 1

**Action: Restrat the kernel\!**

### 2.2 Configure credentials for WASTON OpenScale 
Configure credentials for WASTON OpenScale into the authenticator, which will be used in OpenScale client

In [ ]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator

WOS_CREDENTIALS = {
    "url": "<cluster-url>",
    "username": "<username>",
    "password": "<password>",
    "instance_id": "<openscale instance id>"
}

authenticator = CloudPakForDataAuthenticator(
    url=WOS_CREDENTIALS["url"],
    username=WOS_CREDENTIALS["username"],
    password=WOS_CREDENTIALS["password"],
    disable_ssl_verification=True
)

### 2.3 Configure credentials for IBM Analytics Engine - Spark

Make sure that the Apache Spark manager on IBM Analytics Engine is running, and then provide the following details:

- IAE_SPARK_DISPLAY_NAME: _Display Name of the Spark instance in IBM Analytics Engine_
- IAE_SPARK_JOBS_ENDPOINT: _Spark Jobs Endpoint for IBM Analytics Engine_
- IBM_CPD_VOLUME: _IBM Cloud Pak for Data storage volume name_
- IBM_CPD_USERNAME: _IBM Cloud Pak for Data username_
- IBM_CPD_APIKEY: _IBM Cloud Pak for Data API key_
- IAE_INSTANCE_ID: _IBM Analytics Engine spark instance id_

In [ ]:
IAE_SPARK_DISPLAY_NAME = "<spark-engine-name>"
IAE_SPARK_JOBS_ENDPOINT = "<spark-job-endpoint-for-ibm-analytics-engine>"
IBM_CPD_VOLUME = "<ibm-cpd-volume>"
IBM_CPD_USERNAME = "<ibm-cloud-pak-for-data-username>"
IBM_CPD_APIKEY = "<ibm-cloud-pak-for-data-apikey>"
IAE_INSTANCE_ID = "<ibm-analytics-engine-spark-instance-id>"

The credential informations will be used to run spark job.

In [ ]:
credentials = {
            "connection": {
                "endpoint": IAE_SPARK_JOBS_ENDPOINT,
                "location_type": "cpd_iae",
                "display_name": IAE_SPARK_DISPLAY_NAME,
                "instance_id": IAE_INSTANCE_ID,
                "volume": IBM_CPD_VOLUME
            },
            "credentials": {
                "username": IBM_CPD_USERNAME,
                "apikey":IBM_CPD_APIKEY
            }
        }

### 2.4 Configure resource setting for Spark job

To configure how much of your Spark Cluster resources this job can consume, edit the following values:

- max_num_executors: _Maximum Number of executors to launch for this session_
- min_executors: _Minimum Number of executors to launch for this session_
- executor_cores: _Number of cores to use for each executor_   
- executor_memory: _Amount of memory (in GBs) to use per executor process_
- driver_cores: _Number of cores to use for the driver process_
- driver_memory: _Amount of memory (in GBs) to use for the driver process_

These informations will be configured into spark job parameters.

In [ ]:
spark_parameters = {
                "max_num_executors": 4,
                "min_executors": 1,
                "executor_cores": 1,
                "executor_memory": 1,
                "driver_cores": 1,
                "driver_memory": 1
}

### 2.5 Configure storage parameters for Spark job

In this notebook, DB2 is used as the storage source of input datasets. 
#### Storage Inputs

Please enter a name and description for your JDBC Storage

- JDBC_CONNECTION_NAME: _Custom display name for the JDBC Storage Connection_
- JDBC_CONNECTION_DESCRIPTION: _Custom description for the JDBC Storage Connection_

To connect to your JDBC storage, you must provide the following details:

 - JDBC_HOST: Hostname of the JDBC Connection
 - JDBC_PORT: Port of the JDBC Connection
 - JDBC_USE_SSL: Boolean Flag to indicate whether to use SSL while connecting.
 - JDBC_SSL_CERTIFICATE: SSL Certificate [Base64 encoded string] of the JDBC Connection. Ignored if JDBC_USE_SSL is False.
 - JDBC_DRIVER: Class name of the JDBC driver to use to connect.
 - JDBC_USERNAME: Username of the JDBC Connection
 - JDBC_PASSWORD: Password of the JDBC Connection
 - JDBC_DATABASE_NAME: Name of the Database to connect to.

In [ ]:
JDBC_HOST = "<Hostname of the JDBC Connection>"
JDBC_PORT = "<Port of the JDBC Connection>"
JDBC_USE_SSL = "<Boolean Flag to indicate whether to use SSL while connecting.>"
JDBC_SSL_CERTIFICATE = "<SSL Certificate [Base64 encoded string] of the JDBC Connection. Ignored if JDBC_USE_SSL is False.>"
JDBC_DRIVER = "<Class name of the JDBC driver to use to connect.>"
JDBC_USERNAME = "<Username of the JDBC Connection>"
JDBC_PASSWORD = "<Password of the JDBC Connection>"
JDBC_DATABASE_NAME = "<Name of the Database to connect to.>"

jdbc_url = "jdbc:db2://{}:{}/{}".format(JDBC_HOST, JDBC_PORT, JDBC_DATABASE_NAME)

These informations will be configrued into storage_details session of spark job parameters.

In [ ]:
storage_details = {
    "type": "jdbc",
    "connection": {
        "jdbc_driver": JDBC_DRIVER,
        "jdbc_url": jdbc_url,
        "use_ssl": JDBC_USE_SSL,
        "certificate": JDBC_SSL_CERTIFICATE,
        "location_type": "jdbc"
    },
    "credentials":{
        "username": JDBC_USERNAME,
        "password": JDBC_PASSWORD,
    }
}

#### Training table metadata

Each quality monitor could use its own training table from the database, so the table metadata will be addressed respectively.

### 2.6 Setup OpenScale client 
Setup a Python OpenScale client with above setting.

In [ ]:
from ibm_watson_openscale import APIClient as OpenScaleAPIClient

client = OpenScaleAPIClient(
    service_url=WOS_CREDENTIALS['url'],
    service_instance_id=WOS_CREDENTIALS["instance_id"],
    authenticator=authenticator
)

client.version

## 3. Statistical Parity Difference <a name="spd"></a>

**Statistical Parity Difference** is a fairness metric that can be used to describe the fairness for the model predictions.
It is the difference between the ratio of favourable outcomes in unprivileged and privileged groups. It can
be computed from either the input dataset or the dataset output from a classifier (predicted dataset). A value
of 0 implies both groups have equal benefit, a value less than 0 implies higher benefit for the privileged group, and a value greater than 0 implies higher benefit for the unprivileged group.<br>
$$𝑃(𝑌=1|𝐷=unprivileged)−𝑃(𝑌=1|𝐷=privileged)$$

Take the German credit risk datasets as example, if user set
+ privileged group as Sex="male" 
+ unprivileged group as Sex="female"

and set
+ favourable label as Risk="No Risk"
+ unfavourable label as Risk="Risk"

then, the SPD result 
+ spd > 0 means the unpriviliage group Sex="female" has higher rate to be marked as favourable label "No Risk" than priviliage group Sex="male".
+ spd = 0 means the unpriviliage group Sex="female" has same rate to be marked as favourable label "No Risk" with priviliage group Sex="male".
+ spd < 0 means the unpriviliage group Sex="female" has lower rate to be marked as favourable label "No Risk" than priviliage group Sex="male".


### 3.1 Prepare input for Statistical Parity Difference
The quality monitor stores metadata in the training table.

- TRAIN_SCHEMA_NAME: _Schema name where training table is present_
- TRAIN_TABLE_NAME: _Name of the training table_

In [ ]:
TRAIN_SCHEMA_NAME = "***"
TRAIN_TABLE_NAME = "***"

These informations will be configured into tables session of spark job paramters.

In [ ]:
tables = [
    {
        "database": JDBC_DATABASE_NAME,
        "schema": TRAIN_SCHEMA_NAME,
        "table": TRAIN_TABLE_NAME,
        "type": "training"
    }
]

### 3.2  Statistical Parity Difference Configurations

Setup configuration to compute *Statistical Parity Difference*,<br/>

Configure label and problem type in the overall section.
- **problem_type(str)**: `binary` and `multi-classification` is supported.
- **label_column(str)**: Column name of label in the data frame

Inside `fairness` as below, there are three sections which is required to configure.
- **metrics_configuration(dict)**: Configure *Statistical Parity Difference* as one of the metrics with name `FairnessMetricType.SPD.value`, and it requires a `features` property to describe which features the metric will be computed upon. *Statistical Parity Difference* is supported to run with individual features (eg. `[["a"],["b"]]`), but not suppored to run with intersectional features (eg. `[["a", "b"]]`).

- **protected_attributes(list)**: Describe privileged group defintion for features upon which this metric will be computed. Configure each feature with below information:
  - feature(str): Name of the feature, which should be same as configured in `features` of `metrics_configuration` section.
  - reference_group(list): List of feature values which make a sample privileged. 

- **favourable_label(list)**: A list of favourable labels or outcomes of the model.


**Note** that `label_column` used here is the new added `pred` column.<br/>

In [ ]:
from ibm_metrics_plugin.common.utils.constants import FairnessMetricType, MetricGroupType
spd_config = {
            "problem_type": "***",
            "label_column" : "***",
            "fairness": {
                            "metrics_configuration": {
                                FairnessMetricType.SPD.value: {
                                    "features": [ ["***"] ]                                
                                }
                            },
                            "protected_attributes": [
                                {
                                    "feature": "***",
                                    "reference_group": ["***"]
                                }
                            ],
                            "favourable_label": ["***"]
                        }
        }

spd_config

### 3.3 Compute Statistical Parity Difference
IAE credentials and spark job parameters will be used here.

update all configurations into the spark job parameters

In [ ]:
job_params = {
            "spark_settings": spark_parameters,
            "arguments": {
                "monitoring_run_id": "my_monitoring_run_id",
                "storage": storage_details,
                "tables": tables,
                "metric_configuration": spd_config
            }
        }

job_params

In [ ]:
metrics = client.ai_metrics.compute_metrics_as_job(credentials, job_params)

### 3.4 Check Statistical Parity Difference result
The feature and its statistical parity difference value will be stored as a pair under FairnessMetricType.

In [ ]:
metrics

## 4. Smoothed Empirical Differential<a name="sed"></a>

**Smoothed Empirical Differential(SED)** is a fairness metric that can be used to describe the fairness for the model predictions. It is used to quantify the differential in the probability of favorable/unfavorable outcomes between intersecting groups divided by features. All intersecting groups are equal, there is no unprivileged or privileged groups. 

SED value is the minimum ratio of Dirichlet smoothed probability of favorable and unfavorable outcomes between different intersecting groups in the dataset. Its value is between 0 and 1, excluding 0 and 1. The bigger, the better.

Take the German credit risk datasets as example, assume:

+ the favorable outcomes of label column is "No Risk",
+ the unfavorable outcomes of label column is "Risk".

if user divide dataset by *feature "Sex"*，there will be two intersecting groups:
+ intersecting group Sex="male" 
+ intersecting group Sex="female"

and assume:

+ the Dirichlet smoothed probability of favorable outcomes "No Risk" in intersecting group "Sex"="male" is 0.2
+ the Dirichlet smoothed probability of unfavorable outcomes "Risk" in intersecting group "Sex"="male" is 0.8
+ the Dirichlet smoothed probability of favorable outcomes "No Risk" in intersecting group "Sex"="female" is 0.4
+ the Dirichlet smoothed probability of unfavorable outcomes "Risk" in intersecting group "Sex"="female" is 0.6

then, calculate the label differential between intersecting groups (*Note that it always chooses the smaller one as the numerator or the bigger one as the denominator*): 

+ the favorable outcomes' differential between intersecting group "Sex"="male" and "Sex"="female" will be 0.2/0.4=0.5
+ the unfavorable outcomes' differential between intersecting group "Sex"="male" and "Sex"="female" will be 0.6/0.8=0.75

then, calculate the differential between intersecting groups:
+ the differential between intersecting group "Sex"="male" and "Sex"="female" will be min(0.5, 0.75)=0.5

Since there are only two intersecting groups, so,

+ the final differentials of dataset will be 0.5.

*References: James R. Foulds, Rashidul Islam, Kamrun Naher Keya, Shimei Pan, "An Intersectional Definition of Fairness", Department of Information Systems, University of Maryland, Baltimore County, USA*


### 4.1 Prepare input for Smoothed Empirical Differential 
update the schema and table for Smoothed Empirical Differential in spark job parameters

In [ ]:
TRAIN_SCHEMA_NAME = "***"
TRAIN_TABLE_NAME = "***"

In [ ]:
tables = [
    {
        "database": JDBC_DATABASE_NAME,
        "schema": TRAIN_SCHEMA_NAME,
        "table": TRAIN_TABLE_NAME,
        "type": "training"
    }
]

### 4.2 Smoothed Empirical Differential Configurations 

Configure label and problem type in the overall section.
- **problem_type(str)**: `binary` and `multi-classification` is supported.
- **label_column(str)**: Column name of label in the data frame.

Inside `fairness` as below, there are three sections which is required to configure.
- **metrics_configuration(dict)**: Configure *Smoothed Empirical Differential* as one of the metrics with name `FairnessMetricType.SED.value`, and it requires a `features` property to describes which features the metric will be computed upon. *Smoothed Empirical Differential* is supported to run with individual features (eg. `[["a"],["b"]]`) and with intersectional features (eg. `[["a", "b"]]`).

- **protected_attributes(list)**: Describe protected features upon which this metric will be computed. Configure each feature with such information:
  - feature(str): Name of the feature, which should be same as configured in `features` of `metrics_configuration` section.

- **favourable_label(list)**: A list of favourable labels or outcomes of the model.

Update the Smoothed Empirical Differential configurations, which will be configured into spark job parameters

In [ ]:
sed_config = {
            "problem_type":"binary",
            "label_column" : "***",
            "fairness": {
                            "metrics_configuration": {
                                FairnessMetricType.SED.value: {
                                    "features": [ ["***"] ]                                
                                }
                            },
                            "protected_attributes": [
                                {
                                    "feature": "***"
                                }
                            ],
                            "favourable_label": ["***"]
                        }
        }

sed_config

### 4.3 Compuate Smoothed Empirical Differential 
IAE credentials and spark job parameters will be used here.

In [ ]:
job_params = {
            "spark_settings": spark_parameters,
            "arguments": {
                "monitoring_run_id": "my_monitoring_run_id",
                "storage": storage_details,
                "tables": tables,
                "metric_configuration": sed_config
            }
        }

job_params

In [ ]:
metrics = client.ai_metrics.compute_metrics_as_job(credentials, job_params)

### 4.4 Check Smoothed Empirical Differential result
The features and smoothed empirical differential values will be stored as a pair under FairnessMetricType.

In [ ]:
metrics

## 5. Fair Score Transformer <a name="fst"></a>

**Fair Score Transformer** can be used as post-processing technique that transforms probability estimates ( or scores) of `probabilistic binary classication` model with respect to fairness goals like statistical parity or equalized odds. To use **Fair Score Transformer** in OpenScale, you need first train a **Fair Score Transformer** and then use it to transform scores.

*References: D. Wei, K. Ramamurthy, and F. Calmon, "Optimized Score Transformation for Fair Classification", International Conference on Artificial Intelligence and Statistics, 2020.* 

### 5.1 Prepare input for Fair Score Transformer

To train a Fair Score Transformer, below columns in the dataframe will be used:</br>
***estimate column***: contains the estimate values calculated by the trained classification model.</br>
***protected attribute column***: contains the corresponding protected attributes the trained classification model uses.</br>
***label column (optional)***: contains the ground true values of the estimates column. it is not required to train the transformer but required to compute accuray with the trained transformer.</br>

In [ ]:
TRAIN_SCHEMA_NAME = "***"
TRAIN_TABLE_NAME = "***"

ESTIMATE_COLUMN = "***"
PROTECTED_ATTRIBUTE_COLUMN = "***"
LABEL_COLUMN = "***"

Update these informations to spark job parameters

In [ ]:
tables = [
    {
        "database": JDBC_DATABASE_NAME,
        "schema": TRAIN_SCHEMA_NAME,
        "table": TRAIN_TABLE_NAME,
        "type": "training"
    }
]

### 5.2 Fair Score Transformer Configuration

Setup configuration to fit **Fair Score Transformer**. Inside `metrics_configuration` as below, specify the name of the transformer with `FairnessMetricType.FST.value`. To configure it, you need to provide `params` and `features` information as below. This notebook will transform scores with respect to the **Statistical Parity Difference** fairness goal (set `criteria` as `MSP`).

- **params**: Parameters of Fair Score Transformer
  - epsilon (float): Bound on mean statistical parity or mean equalized odds.
  - criteria (str): Optimize for mean statistical parity ("MSP") or mean equalized odds ("MEO").
  - Aprobabilistic (bool): Indicator of whether actual protected attribute values (False) or probabilistic estimates (True) are provided. Default False.
  - iterMax (float): Maximum number of ADMM iterations. Default 1e3.
- **features**: Columns definition in the dataframe
  - probabilities: Column name of probability estimates.
  - protected: Column name of protected attributes.


In [ ]:
columns = {"probabilities": ESTIMATE_COLUMN, "protected": PROTECTED_ATTRIBUTE_COLUMN}
fst_configuration = {
    "fairness": {
        "metrics_configuration": {
            FairnessMetricType.FST.value: {
                "params": {
                    "epsilon": 0.01,
                    "criteria": "MSP",
                    "Aprobabilistic": False,
                    "iterMax": 1e3
                },
                "features": columns
            }
        }     
    }
}

fst_configuration

### 5.3 Fit Fair Score Transformer
IAE credentials and spark job parameters will be used here. the timeout is in seconds.

In [ ]:
job_params = {
            "spark_settings": spark_parameters,
            "arguments": {
                "monitoring_run_id": "my_monitoring_run_id",
                "storage": storage_details,
                "tables": tables,
                "metric_configuration": fst_configuration
            }
        }

job_params

In [ ]:
fst = client.ai_metrics.fit_transformer_as_job(credentials, job_params, timeout=600)

### 5.4 Use the Trained Fair Score Transformer

#### Compute transformed estimates with Fair Score Transformer
Trained transformer can be used to compute new probability estimates and it requires the exactly same columns as fitting phase.</br> 

**Note:** No matter what column name is used for the existing probability estimates, the new probability estimates column will be named as **r_transformed**.

In [ ]:
# SPARK_DF = spark.createDataFrame("***")
# probs_df = fst.predict_proba(spark, SPARK_DF, columns, keep_cols=LABEL_COLUMN)
# probs_df .show()

#### Compute new labels based on transformed estimates with Fair Score Transformer

Trained transformer can also be used to compute new class labels based on transformed probability estimates, and it requires the exactly same columns as fitting phase. 

**Note:** No matter what column name is used for the `label` column, the new class labels column will be named as **r_transformed_thresh**.

In [ ]:
# SPARK_DF = spark.createDataFrame("***")
# predict_df = fst.predict(spark, SPARK_DF, columns, keep_cols=LABEL_COLUMN)
# predict_df.show()

#### Save the trained Fair Score Transformer

In [ ]:
# import pickle
# pickle.dump(fst, open("fst.pkl", "wb"))